In [1]:
import numpy as np 
import pandas as pd
import re
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
credits=pd.read_csv(r'F:\NLP project\tmdb_5000_credits.csv\tmdb_5000_credits.csv',encoding_errors = 'ignore')
movies=pd.read_csv(r'F:\NLP project\tmdb_5000_movies.csv\tmdb_5000_movies.csv',encoding_errors = 'ignore')


In [3]:
credits_column_renamed=credits.rename(columns={"movie_id":"id"})
movies_df_merged=movies.merge(credits_column_renamed,on='id')
movies_clean=movies_df_merged[['id','original_title','overview','label']]


In [4]:
col=movies_clean.columns
res=[]
for c in col:
    d=movies_clean[col].values.tolist()
    res.append(d)


In [5]:
stopwords = []
corpus=[]
movie_tfidfvector=[]
with open("stopword.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.strip("\n").strip())
countlist = []
for i in range(0,1470):
    with open(r"F:\NLP project\movie/"+str(i)+".txt", "r",encoding='utf-8') as f:    
        data = f.read()   
        
    text = re.sub(r"'s", " 's", data)
    text = re.sub(r"'ve", " 've", data)
    text = re.sub(r"n't", " n't", data)
    text = re.sub(r"'re", " 're", data)
    text = re.sub(r"'d", " 'd", data)
    text = re.sub(r"'ll", " 'll", data)
    text = re.sub(r"[^A-Za-z]", " ", data)
    do_lower_case=True
    if do_lower_case:
        text=text.strip().lower()
    tokens = text.split()
    text_words = [token for token in tokens if token not in stopwords]
    corpus.append(text_words)

In [6]:
def read_corpus(corpus):
    for i, line in enumerate(corpus):
        yield gensim.models.doc2vec.TaggedDocument(line, [i])
train_corpus = list(read_corpus(corpus))

In [23]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
movie_vector=[]
for i in range(0,1470):
    movie_vector.append(model.infer_vector(corpus[i]))
movie_vector

[array([-1.5659414e-01,  1.3607018e-01,  5.4248393e-02, -1.5198274e-01,
         2.4489322e-01,  1.6437984e-01, -1.4219849e-01, -1.3725282e-01,
        -4.4141284e-01, -9.2374608e-02,  1.3464049e-03, -9.9097742e-03,
        -3.4158763e-02,  6.8247721e-02, -3.7378948e-02, -2.4846785e-03,
         8.2230769e-02,  6.6602111e-02, -2.4172565e-01, -3.3532813e-01,
         2.0440958e-01,  4.3880820e-02,  2.3403361e-01, -3.3950844e-01,
         2.3408349e-01,  9.3402699e-02, -1.6967030e-01, -1.9795017e-01,
        -1.6734542e-01, -7.8474276e-02,  5.3818308e-02,  7.0407473e-02,
        -8.6748831e-02,  3.0193385e-01, -1.3069825e-01,  5.7608034e-02,
        -7.7493615e-02,  1.6190250e-01, -5.0903454e-02, -4.1404554e-01,
         1.9372879e-01, -1.5669379e-01,  1.3488808e-01,  5.1121168e-02,
         4.8717678e-01, -1.8794532e-01, -1.1663493e-01, -1.6511545e-02,
        -1.0072502e-01,  1.1020990e-04], dtype=float32),
 array([-0.13705282,  0.12156691,  0.06208488, -0.11420717,  0.07891134,
      

In [24]:
indices=pd.Series(movies_clean.index,index=movies_clean.original_title)
def euclideandistance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))

In [25]:
vector=model.infer_vector(corpus[1])
sims = model.docvecs.most_similar([vector],topn=6)
sims

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13596\3011338544.py:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector],topn=6)


[(581, 0.8799703121185303),
 (778, 0.8532388210296631),
 (1341, 0.8294360041618347),
 (1044, 0.8285475969314575),
 (47, 0.8250677585601807),
 (1034, 0.8136732578277588)]

In [26]:
def recommend(a):
    vector=model.infer_vector(corpus[a])
    sims = model.docvecs.most_similar([vector],topn=10)
    movies_indices=[]
    for i in sims:
        movies_indices.append(i[0])
    series=movies_clean.label.iloc[movies_indices]
    return series.values

In [35]:
acclis=[]
for i in range(0,100):
    a=0
    for j in recommend(i):
        if j==movies_clean['label'][i]:
            a=a+1
    accuracy=a/10
    acclis.append(accuracy)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13596\2514842605.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector],topn=10)


In [36]:
acc=sum(acclis)/len(acclis)
print('%.3f' % acc)

0.462
